# Model

Our model begins with a Regression Classifier.

In [68]:
import kagglehub
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

# Regression Classifier.
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Statsmodels.
import statsmodels.api as sm
import pandas as pd

In [ ]:
# Load dataset.
# Download latest version
path = kagglehub.dataset_download("lashagoch/life-expectancy-who-updated")
print("Path to dataset files:", path)

file = "Life-Expectancy-Data-Updated.csv"

# TODO:  Correct features and assign target.
data = pd.read_csv(path + "/" + file)
data.columns
X = data[['Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing']]

y = data['Life_expectancy']

# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Logistic Regression classifier instance.
model = RandomForestRegressor(oob_score=True)

# Train the model.
model.fit(X_train, y_train)

# Make predictions.
y_pred = model.predict(X_test)

# Evaluate the model.
print(model.oob_score_)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy:  {accuracy}')

Path to dataset files: /Users/nathanlonghurst/.cache/kagglehub/datasets/lashagoch/life-expectancy-who-updated/versions/1
0.9963355636181664


## Method 2

Statsmodels

In [70]:
# # Load dataset.
# # TODO:  Correct features and assign target.
# data = pd.DataFrame(data)
# X = data[['feature1', 'feature2']]
# y = data['target']

# # Add a constant (intercept) to the features.
# X = sm.add_constant(X)

# # Create and fit a Logistic Regression model.
# model = sm.Logit(y, X)
# results = model.fit()

# # Print the summary of results.
# print(results.summary())

# # Make predictions (requires a threshold for classification).
# predictions = results.predict(X)
# binary_predictions = (predictions > 0.5).astype(int)
# print(binary_predictions)